# QG Height Tendency Equation

In [ ]:
import warnings
warnings.filterwarnings('ignore', 'numpy.dtype size changed')
warnings.filterwarnings( 'ignore', category=FutureWarning)

from datetime import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.util as cutil
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import metpy.calc as mpcalc
import metpy.constants as mpconstants
from metpy.units import units
import numpy as np
from pyproj import Proj
from scipy.ndimage import gaussian_filter
import xarray as xr

In [ ]:
date = datetime(2019, 3, 13, 12)

# UCAR RDA Archive - Global 1 deg Data
ds = xr.open_dataset('https://rda.ucar.edu/thredds/dodsC/aggregations/g/ds083.2/2/TP')

# NCEP Archive Global 0.5 deg Data
#ds = xr.open_dataset('https://www.ncei.noaa.gov/thredds/dodsC/model-gfs-g4-anl-files-old/'
#                     f'{date:%Y%m}/{date:%Y%m%d}/gfsanl_4_{date:%Y%m%d}_{date:%H}00_000.grb2')

#ds = xr.open_dataset('groundhogs_day_blizzard/GFS_{0:%Y%m%d}_{0:%H}00.nc'.format(date))

lats = ds.lat.sel(lat=slice(60, 10)).values
lons = ds.lon.sel(lon=slice(360-160, 360-50)).values

subset = dict(vertical=500 * units.hPa, time=date, lat=slice(60, 10), lon=slice(360-160, 360-50))
hght_500 = ds['Geopotential_height_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_500 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_500 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
subset['vertical'] = 700 * units.hPa
tmpk_700 = ds['Temperature_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_700 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_700 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
subset['vertical'] = 300 * units.hPa
tmpk_300 = ds['Temperature_isobaric'].metpy.sel(subset).metpy.quantify()
uwnd_300 = ds['u-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()
vwnd_300 = ds['v-component_of_wind_isobaric'].metpy.sel(subset).metpy.quantify()

ntime = ds.Geopotential_height_isobaric.dims[0]
vtime = hght_500[ntime].values.astype('datetime64[ms]').astype('O')

mapcrs = ccrs.LambertConformal(central_longitude=-100, central_latitude=35, standard_parallels=(30, 60))
datacrs = ccrs.PlateCarree()

# Transform Coordinates ahead of time
lons, lats = np.meshgrid(lons, lats)
tlatlons = mapcrs.transform_points(ccrs.PlateCarree(), lons, lats)
clons = tlatlons[:,:,0]
clats = tlatlons[:,:,1]

# For 1 deg data use 20
# For 0.5 deg data use 80
smoothing_passes = 20

hght_500s = mpcalc.smooth_n_point(hght_500, 9, smoothing_passes)
uwnd_500s = mpcalc.smooth_n_point(uwnd_500, 9, smoothing_passes)
vwnd_500s = mpcalc.smooth_n_point(vwnd_500, 9, smoothing_passes)

tmpk_700s = mpcalc.smooth_n_point(tmpk_700, 9, smoothing_passes)
uwnd_700s = mpcalc.smooth_n_point(uwnd_700, 9, smoothing_passes)
vwnd_700s = mpcalc.smooth_n_point(vwnd_700, 9, smoothing_passes)

tmpk_300s = mpcalc.smooth_n_point(tmpk_300, 9, smoothing_passes)
uwnd_300s = mpcalc.smooth_n_point(uwnd_300, 9, smoothing_passes)
vwnd_300s = mpcalc.smooth_n_point(vwnd_300, 9, smoothing_passes)

sigma = 2.0e-6 * units('m^2 Pa^-2 s^-2')
f0 = 1e-4 * units('s^-1')
Rd = mpconstants.Rd

avor_500 = mpcalc.absolute_vorticity(uwnd_500s, vwnd_500s)
vortadv_500 = mpcalc.advection(avor_500, uwnd_500s, vwnd_500s)

term_A = (f0 * vortadv_500.metpy.unit_array).to_base_units()

tadv_700 = mpcalc.advection(tmpk_700s, uwnd_700s, vwnd_700s).metpy.unit_array
tadv_300 = mpcalc.advection(tmpk_300s, uwnd_300s, vwnd_300s).metpy.unit_array

diff_tadv = ((Rd/(700 * units.hPa)*tadv_700 - Rd/(300 * units.hPa)*tadv_300)/(400 * units.hPa)).to_base_units()

term_B = (-f0**2/sigma*diff_tadv).to_base_units()

In [ ]:
clevs_700_tmpc = np.arange(-40, 41, 2)
clevs_500_hght = np.arange(0, 8000, 60)
clevs_QGHT = np.arange(-8, 8.5, 0.5)

# For 1 deg data use 4
# For 0.5 deg data use 5
barb_spacing = 4

wind_slice = (slice(None, None, barb_spacing), slice(None, None, barb_spacing))

# 1st image
area = [-130, -72, 20, 55]
fig = plt.figure(1, figsize=(17,15))
ax1 = plt.subplot(111, projection=mapcrs)
ax1.set_extent(area, ccrs.PlateCarree())
ax1.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax1.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax1.contourf(clons, clats, -term_A*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax1.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

cs2 = ax1.contour(clons, clats, avor_500*1e5, np.arange(-10, 50, 1), linestyles='dotted',
                  colors='black', alpha=0.75)
plt.clabel(cs2, fmt='%d')

ax1.barbs(lons[wind_slice], lats[wind_slice],
          uwnd_500s[wind_slice].metpy.convert_units('kt').values,
          vwnd_500s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted QGHT local Abs. Vort. Adv. '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGHT_Vort_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

In [ ]:
# 2nd image
fig = plt.figure(1, figsize=(17,15))
ax2 = plt.subplot(111, projection=mapcrs)
ax2.set_extent(area, ccrs.PlateCarree())
ax2.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax2.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax2.contourf(clons, clats, -term_B*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax2.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

ax2.barbs(lons[wind_slice], lats[wind_slice],
          uwnd_500s[wind_slice].metpy.convert_units('kt').values,
          vwnd_500s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted QGHT diff. Temp. Adv. '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGHT_Temp_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()

In [ ]:
# 3rd image
fig = plt.figure(1, figsize=(17,15))
ax3 = plt.subplot(111, projection=mapcrs)
ax3.set_extent(area, ccrs.PlateCarree())
ax3.add_feature(cfeature.COASTLINE.with_scale('50m'))
ax3.add_feature(cfeature.STATES.with_scale('50m'))

cf = ax3.contourf(clons, clats, -(term_A+term_B)*1e13, clevs_QGHT, cmap=plt.cm.bwr, extend='both')
plt.colorbar(cf, orientation='horizontal', pad=0, aspect=50, extendrect=True)

cs = ax3.contour(clons, clats, hght_500s, clevs_500_hght, colors='black')
plt.clabel(cs, fmt='%d')

cs2 = ax3.contour(clons, clats, avor_500*1e5, np.arange(-10, 50, 1), linestyles='dotted',
                  colors='black', alpha=0.75)
plt.clabel(cs2, fmt='%d')

ax3.barbs(lons[wind_slice], lats[wind_slice],
          uwnd_500s[wind_slice].metpy.convert_units('kt').values,
          vwnd_500s[wind_slice].metpy.convert_units('kt').values,
          pivot='middle', color='black', transform=ccrs.PlateCarree())

plt.title('500-hPa GFS Geo. HGHT (m), Inverted Total QGHT (Traditional) '
          '($*10^{13}$ s$^{-3}$) and Wind Barbs (kt)', loc='left')
plt.title(f'Valid Time: {vtime}', loc='right')

plt.savefig(f'QGHT_Total_{date:%Y%m%d_%H}00.png', bbox_inches='tight')
plt.show()